In [1]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/29 17:16:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
#Q1:
spark.version
#3.3.2

'3.3.2'

Question 2:
FHV October 2019

Read the October 2019 FHV into a Spark Dataframe with a schema as we did in the lessons.

Repartition the Dataframe to 6 partitions and save it to parquet.

What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches

In [ ]:
#!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz

In [8]:
fhv = spark.read.csv('./data/fhv_tripdata_2019-10.csv.gz', header=True, inferSchema=True)

In [10]:
fhv.repartition(6).write.parquet('./data/parquet/fhv', mode='overwrite')

In [12]:
!du -ah ./data/parquet/fhv | sort -rh
#6.4M

39M	./data/parquet/fhv
6.4M	./data/parquet/fhv/part-00005-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
6.4M	./data/parquet/fhv/part-00004-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
6.4M	./data/parquet/fhv/part-00003-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
6.4M	./data/parquet/fhv/part-00002-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
6.4M	./data/parquet/fhv/part-00001-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
6.4M	./data/parquet/fhv/part-00000-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet
52K	./data/parquet/fhv/.part-00005-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet.crc
52K	./data/parquet/fhv/.part-00004-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet.crc
52K	./data/parquet/fhv/.part-00003-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet.crc
52K	./data/parquet/fhv/.part-00002-ed315cf3-c371-4733-b471-de102b04254b-c000.snappy.parquet.crc
52K	./data/parquet/fhv/.part-00001-ed315c

Question 3:
Count records

How many taxi trips were there on the 15th of October?

In [13]:
fhv.show(3)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|         264|         264|   null|                B00009|
|              B00013|2019-10-01 00:11:29|2019-10-01 00:13:22|         264|         264|   null|                B00013|
|              B00014|2019-10-01 00:11:43|2019-10-01 00:37:20|         264|         264|   null|                B00014|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
only showing top 3 rows



In [19]:
fhv.filter((fhv.pickup_datetime >= '2019-10-15 00:00:00') & (fhv.pickup_datetime < '2019-10-16 00:00:00')).count()
#62610 trips

62610

In [30]:
from pyspark.sql.functions import col, unix_timestamp, max

Question 4:
Longest trip for each day

What is the length of the longest trip in the dataset in hours?

In [27]:
fhv_duration = fhv.withColumn('pickup_seconds', unix_timestamp(col('pickup_datetime')))
fhv_duration = fhv_duration.withColumn('dropoff_seconds', unix_timestamp(col('dropOff_datetime')))
fhv_duration = fhv_duration.withColumn('duration', (fhv_duration.dropoff_seconds - fhv_duration.pickup_seconds)/3600)

In [32]:
fhv_duration.select(max('duration')).show()
#631 152.5 hours (weird, but ok)

+-------------+
|max(duration)|
+-------------+
|     631152.5|
+-------------+



In [35]:
#I wanted to practice querying in spark sql, so:
fhv_duration.createTempView('duration_table')

In [36]:
spark.sql('''
SELECT duration
FROM duration_table
ORDER BY duration DESC
LIMIT 1
''').show()

+--------+
|duration|
+--------+
|631152.5|
+--------+



Question 5:
User Interface

Spark’s User Interface which shows the application's dashboard runs on which local port?

In [37]:
#PORT 4040

Question 6:
Least frequent pickup location zone

Load the zone lookup data into a temp view in Spark

Using the zone lookup data and the FHV October 2019 data, what is the name of the LEAST frequent pickup location Zone?

In [ ]:
#!wget -P ./data https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv

In [42]:
zones = spark.read.csv('./data/taxi_zone_lookup.csv', header=True, inferSchema=True)

In [44]:
zones.createTempView('zones_table')

In [49]:
spark.sql("""
SELECT zt.LocationID, zt.Zone, COUNT(dt.pickup_datetime)
FROM duration_table dt JOIN zones_table zt on dt.PUlocationID = zt.LocationID
GROUP BY zt.LocationID, zt.Zone
ORDER BY COUNT(dt.pickup_datetime) ASC
LIMIT 1;
""").show()

+----------+-----------+----------------------+
|LocationID|       Zone|count(pickup_datetime)|
+----------+-----------+----------------------+
|         2|Jamaica Bay|                     1|
+----------+-----------+----------------------+

